# Preparacion de datos

In [22]:
import gensim
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from unidecode import unidecode
import itertools
import warnings
warnings.filterwarnings("ignore")

In [7]:
datos = pd.read_csv('../items_titles.csv')
test = pd.read_csv('../items_titles_test.csv')

In [8]:
#Carga de librerias
import os, re, json
from nltk.tokenize import RegexpTokenizer

In [9]:
def quitar_numeros(texto):
    return re.sub("([0-9])", "",texto)

def pasar_a_minusculas(texto):
    return texto.lower()

def tokenizar_palabras(texto):
    tokenizer = RegexpTokenizer(r'\w+')
    return tokenizer.tokenize(texto)

def remove_accents_word(texto):
    return unidecode(texto)

def dropeo_palabras(lista,n=2):
    [lista.remove(x) for x in lista if len(x)<n]
    [lista.remove(x) for x in lista if len(x)<n]
    [lista.remove(x) for x in lista if len(x)<n]
    return lista

In [10]:
def pipeline_data_prep_nlp(texto):
    texto = quitar_numeros(texto)
    texto = pasar_a_minusculas(texto)
    texto = remove_accents_word(texto)
    texto = tokenizar_palabras(texto)
    texto = dropeo_palabras(texto)
    return texto

In [13]:
datos['docs'] = datos['ITE_ITEM_TITLE'].apply(lambda x: pipeline_data_prep_nlp(x))

In [14]:
test['docs'] = test['ITE_ITEM_TITLE'].apply(lambda x: pipeline_data_prep_nlp(x))

In [ ]:
# Esto es para separar de forma manual segun los productos encontrados - no se corre por ahora

listado_test = []
for x in test['docs']:
    if 'tv' in x:
        listado_test.append(1)
    elif 'bicicleta' in x:
        listado_test.append(2)
    elif 'tenis' in x or 'sapatenis' in x or 'sapatilha' in x or 'sapato' in x or 'botinho' in x:
        listado_test.append(3)
    else:
        listado_test.append(0)
        
datos['filtro'] = listado

# Modelo

In [15]:
model = gensim.models.Word2Vec(datos['docs'].to_list())

In [16]:
def calculate_average_vector(tokens, model):
    vectors = [model.wv[token] for token in tokens if token in model.wv]
    if vectors:
        return np.mean(vectors,axis=0)
    else:
        return None

In [17]:
def CosineSimilarity(vector, otherPhraseVec):
    cosine_similarity = np.dot(vector, otherPhraseVec) / (np.linalg.norm(vector) * np.linalg.norm(otherPhraseVec))
    return cosine_similarity

In [24]:
def similitud(datos,model):
    Comparacion = pd.DataFrame({'Item1':[],'Item2':[],'Score Similitud':[]})
    for a, b in itertools.combinations(list(datos.index), 2):
        try:
            simil = CosineSimilarity(calculate_average_vector(datos.iloc[a]['docs'],model),
                                     calculate_average_vector(datos.iloc[b]['docs'],model))
        except TypeError:
            simil = 0
        
        fila = {'Item1':datos['ITE_ITEM_TITLE'].iloc[a],
                'Item2':datos['ITE_ITEM_TITLE'].iloc[b],
               'Score Similitud': simil}
        #if simil > 0.5:
        Comparacion = Comparacion.append(fila, ignore_index=True)
    return Comparacion

In [25]:
resultado = similitud(test,model)

In [26]:
resultado

,Item1,Item2,Score Similitud
0,Tênis Olympikus Esporte Valente - Masculino Kids,Bicicleta Barra Forte Samy C/ 6 Marchas Cubo C...,0.249588
1,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Usthemp Slip-on Temático - Labrador 2,0.487543
2,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Casual Feminino Moleca Tecido Tie Dye,0.697491
3,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Star Baby Sapatinho Conforto + Brinde,0.794490
4,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Oakley Frequency 3.0 Preto/marrom,0.829640
...,...,...,...
1327,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Airr-maxx 95 Tn Slip Mid Totalmente Orig...,0.809466
1328,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Academia Super Confortável Barato Feminino,0.912450
1329,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Fila Disruptor Ii Premium Feminino,0.898497
1330,Tênis Olympikus Esporte Valente - Masculino Kids,Tênis Fila Original,0.854728
